In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
from sklearn.preprocessing import LabelEncoder
import plotly.express as px

In [ ]:

data=pd.read_csv('happiness_score_dataset.csv')
df=pd.DataFrame(data)
df.head()

In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isnull()

In [ ]:
sns.heatmap(df.isnull())

In [ ]:
df.isnull().sum()

In [ ]:
le=LabelEncoder()

data=pd.DataFrame({'Region':['Western Europe','North America','Australia and New Zealand','Middle East and Northern Africa','Latin America and Caribbean','Southeastern Asia','Central and Eastern Europe','Eastern Asia','Sub-Saharan Africa','Southern Asia']})
data['Region_encoded']=le.fit_transform(data['Region'].values)

data=data.drop_duplicates('Region')
print(data)

In [ ]:
df['Region_encoded']=le.fit_transform(df['Region'].values)
df

In [ ]:
df=df.drop(['Country','Region','Happiness Rank'],axis=1)
df

In [ ]:
df.shape

In [ ]:
sns.scatterplot(x=df['Economy (GDP per Capita)'],y=df['Happiness Score'],data=df)
plt.title('Economy (GDP per Capita) vs Happiness Score')
plt.show()


In [ ]:
sns.scatterplot(x=df['Family'],y=df['Happiness Score'],data=df)
plt.title('Family vs Happiness Score')
plt.show()

In [ ]:
sns.scatterplot(x=df['Region_encoded'],y=df['Happiness Score'],data=df)
plt.title('Region_encoded vs Happiness Score')
plt.show()

In [ ]:
dfcorr=df.corr()
dfcorr

In [ ]:
plt.figure(figsize=(20,15))
sns.heatmap(dfcorr,annot=True)

In [ ]:
df.shape

In [ ]:
collist=df.columns.values
ncol=10
nrows=10

plt.figure(figsize=(2*ncol,5*ncol))
for i in range(1,len(collist)):
    plt.subplot(nrows,ncol,i+1)
    sns.boxplot(df[collist[i]],color='green',orient='h')
    plt.tight_layout()

In [ ]:
df.skew()

In [ ]:
sns.distplot(df['Standard Error'])
plt.show()

In [ ]:
sns.distplot(df['Generosity'])
plt.show()

In [ ]:
SElog=np.log(df['Standard Error'])
df['Standard Error']=SElog

sns.distplot(df['Standard Error'])
plt.show()

In [ ]:
print(df['Standard Error'].skew())

In [ ]:
Generosity_sqrt=np.sqrt(df['Generosity'])
df['Generosity']=Generosity_sqrt

print(df['Generosity'].skew())

sns.distplot(df['Generosity'])
plt.show()

In [ ]:
df.skew()

In [ ]:
df

In [ ]:
x=df.iloc[:,1:]
x

In [ ]:
y=pd.DataFrame(df.iloc[:,0])
y

In [ ]:
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
best_r_state=0
accu=0

for i in range(30,200):
    train_x,test_x,train_y,test_y=train_test_split(x,y,test_size=0.25,random_state=i)
    mod=LinearRegression()
    mod.fit(train_x,train_y)
    pred_y=mod.predict(test_x)
    tempaccu=r2_score(test_y,pred_y)
    if tempaccu>accu:
        accu=tempaccu
        best_r_state=i

print(f"Best random_state {best_r_state}")

In [ ]:
train_x,test_x,train_y,test_y=train_test_split(x,y,test_size=0.25,random_state=86)
lg=LinearRegression()
lg.fit(train_x,train_y)
pred_y=lg.predict(test_x)

print("Accuracy Score",r2_score(test_y,pred_y))

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

knn=KNeighborsRegressor()
knn.fit(train_x,train_y)
knn_y=knn.predict(test_x)

print("Accuracy Score",r2_score(test_y,knn_y))

In [ ]:
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor,GradientBoostingRegressor

rfr=RandomForestRegressor(n_estimators=100,random_state=43)
abr=AdaBoostRegressor()
gbr=GradientBoostingRegressor()

for val in [rfr,abr,gbr]:
    val.fit(train_x,train_y)
    print(val, r2_score(test_y,val.predict(test_x)))

In [ ]:
from sklearn.svm import SVR
svr=SVR('linear')
svrscores=cross_val_score(svr,x,y,cv=5)
print(svrscores)
print(svrscores.mean(), svrscores.std())

In [ ]:
from sklearn.model_selection import GridSearchCV
svr=SVR()
parameters={'kernel':['linear','rbf','poly'],'C':[1,10]}
clf=GridSearchCV(svr,parameters)
clf.fit(x,y)

In [ ]:
print(clf.best_params_)

In [ ]:
from sklearn.svm import SVR

svr=SVR(kernel='linear',C=10)
svr.fit(train_x,train_y)
svr.score(train_x,train_y)
svrr_y=svr.predict(test_x)
print("Accuracy Score",r2_score(test_y,svrr_y))

In [ ]:
import joblib
joblib.dump(clf.best_estimator_,"worldhappinessreportfinalDTC.pkl")